In [1]:
import geopandas as gpd
import numpy as np

from feature.utils import timing
from parsers.utils import pcloud
from parsers.utils import raster

Hello


In [2]:
# class BufferableGeoDataFrame(gpd.GeoDataFrame):
#     def __init__(self, *args, **kwargs) -> None:
#         super().__init__()
# 
#     def buffer(self, *args, **kwargs) -> None:
#         ...

@timing
def __init__(filename: str) -> gpd.GeoDataFrame:
    return gpd.read_file(filename)


@timing
# TODO: Review the various buffer options.
# TODO: Rewrite this function as an appropriate class method.
def buffer(objs: gpd.GeoDataFrame, dist: float = 10) -> None:
    objs["geometry"] = objs["geometry"].buffer(dist)

## Step 1
Load the point cloud into memory.

In [3]:
pc = pcloud.PointCloud("../aula.LAZ")

func __init__ :-> 261 ms


## Step 2
Load the building footprints into memory.

In [4]:
fps = __init__("C:/Documents/RoofSense/data/temp/9-284-556.buildings.gpkg")

func __init__ :-> 195 ms


## Step 3
Buffer the footprints by 10 m.

NOTE: The buffer distance is selected such that most if not all the natural neighbors of 
      the points along the footprint edges are included in the subsequent steps.

In [5]:
buffer(fps)

func buffer :-> 64 ms


## Step 4
Intersect the point cloud with the buffers.

In [6]:
ids = pc.intersect(fps)

func intersect :-> 32013 ms


In [7]:
pc.las.points = pc.las.points[ids]

## Step 5
Compute the Delaunay triangulation of the remaining points.

In [8]:
pc.triangulate()

func triangulate :-> 10152 ms


## Step 6
 Estimate the slope field of the corresponding network.

TODO

In [9]:
# TOSELF: Promote this argument to an environment variable?
CELL_SIZE = 0.25

grid = raster.Raster(CELL_SIZE, pc.dt.get_bbox())

# FIXME: Integrate this block into the Raster initializer.
# Construct the grid.
# TODO: Add documentation.
rows, cols = np.mgrid[grid.len_y - 1:-1:-1, 0:grid.len_x]
# TODO: Add documentation.
xx = grid.bbox[0] + CELL_SIZE * (cols + 0.5)
yy = grid.bbox[1] + CELL_SIZE * (rows + 0.5)
# TODO: Add documentation.
cells = np.column_stack([xx.ravel(), yy.ravel()])

TODO

In [10]:
# TODO: Speed up this block.
# TOSELF: Compute the slope for all faces to avoid additional calls to dt.locate?
valid_cells = []
valid_faces = []
for i, center in enumerate(cells):
    try:
        valid_faces.append(pc.dt.locate(*center))
    except Exception:
        continue
    valid_cells.append(i)

# TOSELF: Discard duplicate faces?
# valid_faces = np.unique(valid_faces, axis=0)

In [11]:
# TODO: Add documentation.
dt_pts = pc.dt.points
valid_faces = np.array(valid_faces)
v1 = dt_pts[valid_faces[:, 0]]
v2 = dt_pts[valid_faces[:, 1]]
v3 = dt_pts[valid_faces[:, 2]]

u = v2 - v1
v = v3 - v1

n = np.cross(u, v)
n = n / np.linalg.norm(n, axis=1)[:, None]

z = np.array([0, 0, 1])
s = np.degrees(np.arccos(np.dot(n, z)))

In [14]:
grid._Raster__data[np.divmod(valid_cells, grid.len_x)] = s
grid.save("../aula_slpe.tiff")

## Step 6
 Rasterize the slope and reflectance fields of the corresponding network.

In [13]:
# Interpolate the field values at the cell centers.
vals = pc.dt.interpolate({"method": "Laplace"}, cells)

# Populate the raster.
grid._Raster__data = vals.reshape((grid.len_y, grid.len_x))

# Save the raster.
grid.save("../aula_elev.tiff")